# Embeddings  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PragmaticsLab/NLP-course-FinTech/blob/master/seminars/2/2_embeddings.ipynb)

## Word2Vec

Векторные модели, которые мы рассматривали до этого (tf-idf, BOW), условно называются *счётными*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов.

Другой класс моделей, который более повсевмёстно распространён на сегодняшний день, называется *предсказательными* (или *нейронными*) моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей слов. Одной из самых известных таких моделей является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)


Полученные таким образом вектора называются *распределенными представлениями слов*, или **эмбеддингами**.


### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).  

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста.

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова.

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии) и решать примеры:

* *король: мужчина = королева: женщина*
 $\Rightarrow$
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

### Проблемы
Невозможно установить тип семантических отношений между словами: синонимы, антонимы и т.д. будут одинаково близки, потому что обычно употребляются в схожих контекстах. Поэтому близкие в векторном пространстве слова называют *семантическими ассоциатами*. Это значит, что они семантически связаны, но как именно — непонятно.


## RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка. Также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятором семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл
* .bin.gz — бинарник

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`.

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2018 г.

In [1]:
import re
import gensim
import logging
import nltk.data
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
#urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")
urllib.request.urlretrieve("https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz", "ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz")

('ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz',
 <http.client.HTTPMessage at 0x7e943150e3e0>)

In [9]:
#model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'
model_path = 'ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=False)

In [14]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'биткоин_NOUN']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [15]:
for word in words:
    # есть ли слово в модели?
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_NOUN
[-0.028439 -0.037945  0.011242 -0.021425  0.057268  0.071456 -0.001182
  0.026486 -0.033046 -0.019704]
неделя_NOUN :  0.7097942233085632
месяц_NOUN :  0.6908209919929504
утро_NOUN :  0.6243225932121277
днемя_NOUN :  0.6224159598350525
днями_NOUN :  0.6100884675979614
воскресенье_NOUN :  0.6087555885314941
вечер_NOUN :  0.6030082702636719
дня_NOUN :  0.592153012752533
час_NOUN :  0.590473473072052
сутки_NOUN :  0.5869504809379578


ночь_NOUN
[ 0.046085  0.00211   0.065384  0.010745 -0.035544  0.053312  0.084506
  0.005005 -0.037686 -0.006833]
ночь_PROPN :  0.7704508304595947
вечер_NOUN :  0.7683228254318237
утро_NOUN :  0.7520124316215515
полночь_NOUN :  0.7201331853866577
рассвет_NOUN :  0.6792924404144287
полдень_NOUN :  0.6637035012245178
утро_PROPN :  0.6531521677970886
ночь_ADV :  0.6248846650123596
сумерки_NOUN :  0.6227153539657593
ночью_NOUN :  0.6219336986541748


человек_NOUN
[-0.018347  0.00314   0.100693 -0.020991 -0.028008  0.075846 -0.105468
 -0.043332 -0.042502 

Находим косинусную близость пары слов:

In [16]:
print(model_ru.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.32542086


Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [17]:
print(model_ru.most_similar(positive=['пицца_NOUN', 'сибирь_NOUN'], negative=['италия_NOUN'])[0][0])

пельмень_NOUN


In [18]:
model_ru.doesnt_match('пицца_NOUN пельмень_NOUN хот-дог_NOUN ананас_NOUN'.split())

'ананас_NOUN'

### Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle).

In [19]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv

--2023-09-19 18:51:16--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67281491 (64M) [text/plain]
Saving to: ‘unlabeledTrainData.tsv’

unlabeledTrainData. 100%[===================>]  64.16M   211MB/s    in 0.3s    

2023-09-19 18:51:19 (211 MB/s) - ‘unlabeledTrainData.tsv’ saved [67281491/67281491]



In [20]:
data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

len(data)

50000

In [21]:
data.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приводим все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов. Здесь используется токенизатор из библиотеки `nltk`.

In [22]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [23]:
def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [24]:
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []

print("Parsing sentences from training set...")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


<ipython-input-23-a616f10ffbb7>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review, "lxml").get_text()


In [25]:
print(len(sentences))
print(sentences[0])

529416
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


In [26]:
# это понадобится нам позже

with open('clean_text.txt', 'w') as f:
    for s in sentences[:5000]:
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель.


Основные параметры:
* данные должны быть итерируемым объектом
* vector_size — размер вектора,
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение превышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [28]:
print("Training model...")

%time model_en = word2vec.Word2Vec(sentences, workers=4, vector_size=300, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 4min 12s, sys: 1.23 s, total: 4min 13s
Wall time: 2min 32s


Смотрим, сколько в модели слов.

In [31]:
print(len(model_en.wv.key_to_index))

28308


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [30]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

[('actress', 0.7577528953552246)]
[('men', 0.6334580779075623)]
[('europe', 0.726995587348938), ('australia', 0.6912207007408142), ('national', 0.6908457279205322)]
novel


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейся модели, чтобы потом сравнить результат с дообученной.

In [32]:
model_en.wv.similarity('lion', 'rabbit')

0.22788347

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [33]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt

--2023-09-19 19:07:23--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167631 (164K) [text/plain]
Saving to: ‘alice.txt’

alice.txt           100%[===================>] 163.70K  --.-KB/s    in 0.02s   

2023-09-19 19:07:23 (10.1 MB/s) - ‘alice.txt’ saved [167631/167631]



In [34]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)

print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель, а `KeyedVectors` — нельзя. Поэтому сохранять модель нужно в соответствующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [35]:
model_path = "movie_reviews.model"

print("Saving model...")
model_en.save(model_path)

Saving model...


In [36]:
model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

(96997, 150225)

Лев и кролик стали... дальше друг от друга!

In [37]:
model.wv.similarity('lion', 'rabbit')

0.22345278

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1.

Кроме того, сохраним не полные вектора, а `KeyedVectors`.

In [38]:
model.init_sims(replace=True)
model_path = "movies_alice.bin"

print("Saving model...")
model_en.wv.save_word2vec_format(model_path, binary=True)

<ipython-input-38-b59eacbacf4e>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Saving model...


## Оценка

Это, конечно, хорошо, но как понять, какая модель лучше? Или вот, например, я сделал свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости.

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость |
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификаторов мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующим образом:

| слово 1    | слово 2    | отношение     |
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия), получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой.

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [39]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt

--2023-09-19 19:20:44--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 871776 (851K) [text/plain]
Saving to: ‘ru_analogy_tagged.txt’

ru_analogy_tagged.t 100%[===================>] 851.34K  --.-KB/s    in 0.03s   

2023-09-19 19:20:44 (30.4 MB/s) - ‘ru_analogy_tagged.txt’ saved [871776/871776]



In [41]:
from gensim.test.utils import datapath

#res = model_ru.accuracy('ru_analogy_tagged.txt')
analogy_scores = model.wv.evaluate_word_analogies(datapath('questions-words.txt'))

In [55]:
print(analogy_scores[1])
#print(res[4]['incorrect'][:10])

[{'section': 'capital-common-countries', 'correct': [('ATHENS', 'GREECE', 'LONDON', 'ENGLAND'), ('ATHENS', 'GREECE', 'PARIS', 'FRANCE'), ('ATHENS', 'GREECE', 'TOKYO', 'JAPAN'), ('BAGHDAD', 'IRAQ', 'BERLIN', 'GERMANY'), ('BAGHDAD', 'IRAQ', 'MOSCOW', 'RUSSIA'), ('BANGKOK', 'THAILAND', 'BERLIN', 'GERMANY'), ('BANGKOK', 'THAILAND', 'TOKYO', 'JAPAN'), ('BEIJING', 'CHINA', 'TOKYO', 'JAPAN'), ('BERLIN', 'GERMANY', 'BAGHDAD', 'IRAQ'), ('HANOI', 'VIETNAM', 'TOKYO', 'JAPAN'), ('HANOI', 'VIETNAM', 'BAGHDAD', 'IRAQ'), ('HAVANA', 'CUBA', 'PARIS', 'FRANCE'), ('HELSINKI', 'FINLAND', 'BERLIN', 'GERMANY'), ('LONDON', 'ENGLAND', 'PARIS', 'FRANCE'), ('LONDON', 'ENGLAND', 'TOKYO', 'JAPAN'), ('MADRID', 'SPAIN', 'PARIS', 'FRANCE'), ('MOSCOW', 'RUSSIA', 'PARIS', 'FRANCE'), ('MOSCOW', 'RUSSIA', 'TOKYO', 'JAPAN'), ('MOSCOW', 'RUSSIA', 'BERLIN', 'GERMANY'), ('MOSCOW', 'RUSSIA', 'LONDON', 'ENGLAND'), ('PARIS', 'FRANCE', 'TOKYO', 'JAPAN'), ('PARIS', 'FRANCE', 'BERLIN', 'GERMANY'), ('PARIS', 'FRANCE', 'LONDON', 'E

## Визуализация

На полученную модель можно посмотреть, визуализировав ее, например, на плоскости.
### t-SNE

**t-SNE**  (*t-distributed Stochastic Neighbor Embedding*) — техника нелинейного снижения размерности и визуализации многомерных переменных. Она разработана специально для данных высокой размерности Л. ван дер Маатеном и Д. Хинтоном, [вот их статья](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). t-SNE — это итеративный алгоритм, основанный на вычислении попарных расстояний между всеми объектами (в том числе поэтому он довольно медленный).


Изобразим на плоскости 1000 самых частотных слов из коллекции текстов про кино:

In [56]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []


fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)

for w in fd.most_common(1000):
    top_words.append(w[0])

print(top_words[:50:])
top_words_vec = model[top_words]

<ipython-input-56-c8c3c09dbfcf>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm(sentences):


  0%|          | 0/529416 [00:00<?, ?it/s]

['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'with', 'for', 'movie', 'but', 'film', 'you', 't', 'on', 'not', 'he', 'are', 'his', 'have', 'be', 'one', 'all', 'they', 'at', 'by', 'who', 'an', 'from', 'so', 'like', 'there', 'or', 'her', 'just', 'about', 'out', 'has', 'if', 'what', 'some', 'good', 'can']


TypeError: ignored

In [ ]:
top_words_vec = model[top_words]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


CPU times: user 9.7 s, sys: 193 ms, total: 9.9 s
Wall time: 5.04 s


In [ ]:
 !pip install bokeh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Чтобы вычислить преобразование t-SNE быстрее (и иногда еще и эффективнее), можно сперва снизить размерность исходных данных с помощью, например, SVD, и потом применять t-SNE:

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd_50 = TruncatedSVD(n_components=50)
top_words_vec_50 = svd_50.fit_transform(top_words_vec)
top_words_tsne2 = TSNE(n_components=2, random_state=0).fit_transform(top_words_vec_50)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [ ]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words, +SVD)")

source = ColumnDataSource(data=dict(x1=top_words_tsne2[:,0],
                                    x2=top_words_tsne2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

## FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствующих в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Тьюториал](https://fasttext.cc/docs/en/support.html)
* [Вектора для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Вектора, обученные на википедии](https://fasttext.cc/docs/en/pretrained-vectors.html) (отдельно для 294 разных языков)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

In [57]:
! git clone https://github.com/facebookresearch/fastText.git
! pip3 install fastText/.

Cloning into 'fastText'...
remote: Enumerating objects: 3953, done.
remote: Counting objects: 100% (987/987), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 3953 (delta 881), reused 844 (delta 830), pack-reused 2966
Receiving objects: 100% (3953/3953), 8.26 MiB | 26.34 MiB/s, done.
Resolving deltas: 100% (2513/2513), done.
Processing ./fastText
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4291883 sha256=ad145aebb646578286724ae7a3d49f5693efc206659a1f18b1fb1c41ebe88e08
  Stored in directory: /tmp/pip-ephem-wheel-cache-enrqaat7/wheels/8b/05/af/3cfae069d904597d44b309c956601b611bdf8967bcbe968903
Successfully built fasttext


In [58]:
import fasttext

# так можно обучить свою модель
ft_model = fasttext.train_unsupervised('clean_text.txt', minn=2, maxn=5, dim=300)

In [59]:
ft_model.get_word_vector("movie")

array([ 7.56988811e-06,  1.88115984e-02, -4.40639257e-02, -1.08528741e-01,
        4.68422808e-02,  1.84484899e-01, -6.33455068e-02, -1.27518758e-01,
        2.03700349e-01,  5.55569008e-02,  2.54402850e-02,  3.04404907e-02,
       -4.00295183e-02,  3.37889902e-02,  6.18524943e-03,  1.17644288e-01,
       -1.99238788e-02, -5.66098979e-03,  9.16757286e-02,  1.95100773e-02,
        7.00193718e-02,  1.23193100e-01,  1.33457378e-01, -9.21930361e-04,
        3.60089983e-03,  1.08213328e-01, -3.03807352e-02, -7.69824460e-02,
        9.27532688e-02,  2.30971742e-02, -1.48166597e-01,  3.42830829e-02,
       -6.99323267e-02,  2.10133251e-02, -1.57621399e-01,  2.76132859e-02,
       -8.51107016e-02, -5.21767773e-02,  7.92209804e-02,  3.02974042e-02,
        1.60744637e-01,  1.47662073e-01,  4.48724367e-02,  3.91076691e-02,
       -2.74802856e-02, -1.58282548e-01,  2.40783647e-01, -7.24463686e-02,
       -1.10178925e-01, -6.55114651e-02, -1.04578637e-01,  1.16933562e-01,
       -1.03905864e-01,  

In [60]:
ft_model.get_nearest_neighbors('actor')

[(0.9999454021453857, 'arab'),
 (0.9999423623085022, 'poor'),
 (0.9999375939369202, 'superior'),
 (0.9999333024024963, 'american'),
 (0.9999332427978516, 'amanda'),
 (0.9999306797981262, 'camera'),
 (0.9999271035194397, 'victims'),
 (0.9999265670776367, 'erotic'),
 (0.9999223947525024, 'america'),
 (0.9999216794967651, 'visuals')]

In [61]:
ft_model.get_analogies("woman", "man", "actor")

[(0.9999073147773743, 'doctor'),
 (0.9999064803123474, 'screenwriter'),
 (0.9999054074287415, 'victoria'),
 (0.9999045729637146, 'performed'),
 (0.999900221824646, 'utterly'),
 (0.9998974800109863, 'predator'),
 (0.9998929500579834, 'spectacular'),
 (0.9998927712440491, 'affair'),
 (0.9998879432678223, 'jerry'),
 (0.9998875260353088, 'perfectly')]

In [62]:
# проблема с опечатками решена

ft_model.get_nearest_neighbors('actr')

[(0.9999589323997498, 'actor'),
 (0.9999556541442871, 'america'),
 (0.9999416470527649, 'actors'),
 (0.9999362230300903, 'performance'),
 (0.99993497133255, 'producer'),
 (0.9999319911003113, 'class'),
 (0.9999313354492188, 'barbara'),
 (0.9999285936355591, 'era'),
 (0.9999228715896606, 'perhaps'),
 (0.9999222159385681, 'producers')]

In [64]:
# проблема с out of vocabulary словами - тоже

ft_model.get_nearest_neighbors('moviegeek')

[(0.9999784231185913, 'view'),
 (0.9999722242355347, 'money'),
 (0.9999671578407288, 'massive'),
 (0.9999662041664124, 'negative'),
 (0.9999658465385437, 'notch'),
 (0.9999650120735168, 'spoken'),
 (0.9999611377716064, 'new'),
 (0.9999605417251587, 'sweet'),
 (0.9999600052833557, 'revenge'),
 (0.9999598860740662, 'believable')]

In [65]:
!wget -O positive.csv https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

--2023-09-19 19:50:02--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2023-09-19 19:50:02--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc80990402c93d691e468232a01.dl.dropboxusercontent.com/cd/0/inline/CEBV5BTRCZIRjPNtYEajhXcFvo6H24CulP3OqZeGPTMDxNn9MM4DwdzyK9E-7VKCg0kQWguuiTCDDJz5XzO2w3tc1FQK59Mh6Bww14dXoeOz0BZuQl8j3BnW10Xqalp3qSs/file# [following]
--2023-09-19 19:50:03--  https://ucc80990402c93d691e468232a01.dl.dropboxusercontent.com/cd/0/inline/CEBV5BTRCZIRjPNtYEajhXcFvo6H24CulP3OqZeGPTMDxNn9MM4DwdzyK9E-7VKCg0kQWguuiTCDDJz5XzO2w3tc1FQK59Mh6Bww14dXoeOz0BZuQl8j3B

In [66]:
!wget -O negative.csv https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

--2023-09-19 19:50:08--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2023-09-19 19:50:09--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc1f5102d7ee0866fd14a336481.dl.dropboxusercontent.com/cd/0/inline/CED1hDOqVCVVwXQCIZosi3VFv4zMsHyXG0RrbFhEU6du8fbQq30tCzdcoZs7qw-ipC9pSIumxYXpkgw7i6uxVI0jvJhhZAR86igmbalcVok1829eH5Sem5b9yoenOCIrYSw/file# [following]
--2023-09-19 19:50:09--  https://ucc1f5102d7ee0866fd14a336481.dl.dropboxusercontent.com/cd/0/inline/CED1hDOqVCVVwXQCIZosi3VFv4zMsHyXG0RrbFhEU6du8fbQq30tCzdcoZs7qw-ipC9pSIumxYXpkgw7i6uxVI0jvJhhZAR86igmbalcVok1829eH5Sem5

In [67]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)
df.head()

<ipython-input-67-311ffed6a2f4>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = positive.append(negative)


,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [68]:
len(df)

226834

Проведем стандартный препроцессинг:

In [69]:
! pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=0c5dfe70271e3b9608281442188de9272129e581227b6b0ec86ef90bf264ea33
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [70]:
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

m = pymorphy2.MorphAnalyzer()

regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [71]:
#@lru_cache(maxsize=128)
# если вы работаете на колабе, можно заменить pymorphy на mystem и раскомментировать первую строку про lru_cache
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "

In [72]:
def clean_text(text):
    return lemmatize(words_only(text))

In [73]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))


df['lemmas'] = lemmas
df.head()

<ipython-input-73-f45f36ff0d3c>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))


  0%|          | 0/226834 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

Запишем полученные данные в формате для обучения классификатора:

In [ ]:
X = df.lemmas.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 151978
total test examples 74856


In [ ]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')


with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [ ]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.8984717323928609
R@1: 0.8984717323928609
Number of examples: 74856
